In [222]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [223]:
dirname = './data/'
files = [f for f in os.listdir(dirname) if f.endswith('.csv')]
files

['tuesday.csv', 'monday.csv', 'thursday.csv', 'wednesday.csv', 'friday.csv']

In [224]:
dfs = []
for f in files:
    day = f[:-4]
    df = pd.read_csv(dirname + f, sep=';')
    df['customer_no'] = day + '_' + df['customer_no'].astype(str)
    dfs.append(df)

In [225]:
df = pd.concat(dfs)
df.shape

(24877, 3)

In [278]:
df.head()

location prev_location  dummy
customer_no timestamp                                        
friday_1    2019-09-06 07:00:00    dairy      entrance      1
            2019-09-06 07:01:00    dairy         dairy      1
            2019-09-06 07:02:00    dairy         dairy      1
            2019-09-06 07:03:00    dairy         dairy      1
            2019-09-06 07:04:00   spices         dairy      1

In [227]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
df.head(2)

,customer_no,location
timestamp,,
2019-09-03 07:02:00,tuesday_1,fruit
2019-09-03 07:02:00,tuesday_2,drinks


In [228]:
# Add missing timeslots for each customer & add 'prev_location'
df = df.groupby('customer_no')[['location']].resample(rule='60S').ffill()
df['prev_location'] = df['location'].shift()
df.head()

location prev_location
customer_no timestamp                                 
friday_1    2019-09-06 07:00:00    dairy           NaN
            2019-09-06 07:01:00    dairy         dairy
            2019-09-06 07:02:00    dairy         dairy
            2019-09-06 07:03:00    dairy         dairy
            2019-09-06 07:04:00   spices         dairy

In [270]:
# Clean 'prev_location' column
mask1 = df['prev_location'] == 'checkout'  
mask2 = df['location'] != 'checkout'
df.loc[mask1 & mask2, 'prev_location'] = np.nan
df['prev_location'].fillna('entrance', inplace=True)
df.head(10)

location prev_location  dummy
customer_no timestamp                                         
friday_1    2019-09-06 07:00:00     dairy      entrance      1
            2019-09-06 07:01:00     dairy         dairy      1
            2019-09-06 07:02:00     dairy         dairy      1
            2019-09-06 07:03:00     dairy         dairy      1
            2019-09-06 07:04:00    spices         dairy      1
            2019-09-06 07:05:00  checkout        spices      1
friday_10   2019-09-06 07:06:00     fruit      entrance      1
            2019-09-06 07:07:00     fruit         fruit      1
            2019-09-06 07:08:00     fruit         fruit      1
            2019-09-06 07:09:00     fruit         fruit      1

In [271]:
# Crosstab
TM = pd.crosstab(df['location'], df['prev_location'], normalize=1)
TM.sum()

prev_location
dairy       1.0
drinks      1.0
entrance    1.0
fruit       1.0
spices      1.0
dtype: float64

In [279]:
# Final transition matrix
TM

dummy                                        
prev_location     dairy    drinks  entrance     fruit    spices
location                                                       
checkout       0.102678  0.215505  0.000000  0.201054  0.149888
dairy          0.737032  0.011001  0.287583  0.095924  0.193533
drinks         0.058625  0.598499  0.153296  0.055241  0.163109
fruit          0.050073  0.088012  0.377646  0.597104  0.091271
spices         0.051592  0.086983  0.181475  0.050677  0.402198

In [280]:
# Longer version without crosstab 
df['dummy'] = 1
TM_abs = df.groupby(['location', 'prev_location']).count().unstack()
TM_abs.fillna(0, inplace=True)
TM_abs

dummy                                 
prev_location    dairy  drinks entrance   fruit  spices
location                                               
checkout        1825.0  2096.0      0.0  2555.0   941.0
dairy          13100.0   107.0   2133.0  1219.0  1215.0
drinks          1042.0  5821.0   1137.0   702.0  1024.0
fruit            890.0   856.0   2801.0  7588.0   573.0
spices           917.0   846.0   1346.0   644.0  2525.0

In [281]:
TM = TM_abs.apply(lambda x: x/x.sum(), axis=0)

In [282]:
TM

dummy                                        
prev_location     dairy    drinks  entrance     fruit    spices
location                                                       
checkout       0.102678  0.215505  0.000000  0.201054  0.149888
dairy          0.737032  0.011001  0.287583  0.095924  0.193533
drinks         0.058625  0.598499  0.153296  0.055241  0.163109
fruit          0.050073  0.088012  0.377646  0.597104  0.091271
spices         0.051592  0.086983  0.181475  0.050677  0.402198

In [283]:
TM.sum()

       prev_location
dummy  dairy            1.0
       drinks           1.0
       entrance         1.0
       fruit            1.0
       spices           1.0
dtype: float64